In [46]:
# Imports
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset  # Gives easier dataset managment by creating mini batches etc.
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split

import sys
import time
import copy

In [47]:
pd_xela_allfiles = pd.read_csv('data.csv', index_col=0)
pd_sliplabel_allfiles = pd.read_csv('labels.csv')
pd_sliplabel_allfiles = pd_sliplabel_allfiles['slip']
tac_label = pd_sliplabel_allfiles.values.reshape(pd_sliplabel_allfiles.shape[0], 1)

In [48]:
col_x = []
col_y = []
col_z = []

pd_columns = pd_xela_allfiles.columns
for col in pd_columns:
    if col.endswith('x'):
        col_x.append(col)
    
    elif col.endswith('y'):
        col_y.append(col)
    
    elif col.endswith('z'):
        col_z.append(col)

pd_xela_allfiles_x = pd_xela_allfiles[col_x]
pd_xela_allfiles_y = pd_xela_allfiles[col_y]
pd_xela_allfiles_z = pd_xela_allfiles[col_z]

In [49]:
sc = MinMaxScaler()
sc.fit(pd_xela_allfiles_x)
pd_xela_allfiles_x = sc.transform(pd_xela_allfiles_x)

sc.fit(pd_xela_allfiles_y)
pd_xela_allfiles_y = sc.transform(pd_xela_allfiles_y)

sc.fit(pd_xela_allfiles_z)
pd_xela_allfiles_z = sc.transform(pd_xela_allfiles_z)

In [50]:
pd_x = []
pd_y = []
pd_z = []

for row in range(len(pd_xela_allfiles_x)):
    pd_x.append(np.rot90(pd_xela_allfiles_x[row].reshape(4,4)))
    pd_y.append(np.rot90(pd_xela_allfiles_y[row].reshape(4,4)))
    pd_z.append(np.rot90(pd_xela_allfiles_z[row].reshape(4,4)))

pd_main = [pd_x, pd_y, pd_z]
pd_image = np.zeros( (pd_xela_allfiles.shape[0], 3, 4, 4))


In [51]:
#per row, get (4,4) of x, y, z and assign it to pd_image to form the image
for row in range(pd_xela_allfiles.shape[0]):
    x_4_4 = pd_main[0][row]
    y_4_4 = pd_main[1][row]
    z_4_4 = pd_main[2][row]

    pd_image[row][0] = x_4_4
    pd_image[row][1] = y_4_4
    pd_image[row][2] = z_4_4


In [52]:
# up_size = 4
# n_images = len(pd_x)

# tac_imagex = np.zeros((n_images, up_size, up_size), np.float32)
# tac_imagey = np.zeros((n_images, up_size, up_size), np.float32) 
# tac_imagez = np.zeros((n_images, up_size, up_size), np.float32) 
# #resize image to 3, up_size, up_size

# for row in range(len(pd_x)):
#     img_4_4_x = pd_x[row]
#     tac_imagex[row] = cv2.resize(img_4_4_x.astype(np.float32), dsize=(up_size, up_size), interpolation=cv2.INTER_CUBIC)

# for row in range(len(pd_y)):
#     img_4_4_y = pd_y[row]
#     tac_imagey[row] = cv2.resize(img_4_4_y.astype(np.float32), dsize=(up_size, up_size), interpolation=cv2.INTER_CUBIC)

# for row in range(len(pd_z)):
#     img_4_4_z = pd_z[row]
#     tac_imagez[row] = cv2.resize(img_4_4_z.astype(np.float32), dsize=(up_size, up_size), interpolation=cv2.INTER_CUBIC)

# pd_main = [tac_imagex, tac_imagey, tac_imagez]

# #arrange pd_main in a 3, 224, 224 array where its 3(224, 224) of x, y, z values
# pd_image = np.zeros( (pd_xela_allfiles.shape[0], 3, up_size, up_size))

# #per row, get (224,224) of x, y, z and assign it to pd_image to form the image
# for row in range(pd_xela_allfiles.shape[0]):
#     x_4_4 = pd_main[0][row]
#     y_4_4 = pd_main[1][row]
#     z_4_4 = pd_main[2][row]

#     pd_image[row][0] = x_4_4
#     pd_image[row][1] = y_4_4
#     pd_image[row][2] = z_4_4

# plt.imshow(tac_imagex[100040])
# plt.show()

In [53]:
# up_size = 16
# n_images = len(pd_x)
# tac_image = np.zeros((n_images, 3, up_size, up_size), np.float32) 
# for row in range(n_images):

#     #resize image to 3, up_size, up_size
#     for channel in range(3):
#         image_per_channel = pd_image[row][channel]
#         tac_image[row][channel] = cv2.resize(image_per_channel.astype(np.float32), dsize=(up_size, up_size), interpolation=cv2.INTER_CUBIC)


In [54]:
tac_image = torch.from_numpy(pd_image.astype(np.float32))
tac_label = torch.from_numpy(tac_label.astype(np.float32))

In [55]:
#split the data into train and test
tac_image_train, tac_image_test, tac_label_train, tac_label_test = train_test_split(tac_image, tac_label, test_size=0.1, shuffle=True)

#split into train and validation
tac_image_train, tac_image_valid, tac_label_train, tac_label_valid = train_test_split(tac_image_train, tac_label_train, test_size=0.3, shuffle = True)


In [56]:
class Batch_Taxels(Dataset):
    
    def __init__(self, tac_image_train, tac_label_train, tac_image_valid, tac_label_valid, valid = None):
        self.x = tac_image_train
        self.y = tac_label_train
        self.xvalid = tac_image_valid
        self.yvalid = tac_label_valid
        self.valid = valid

    def __len__(self):
        if self.valid == True:
            return self.xvalid.shape[0]
        else:
            return self.x.shape[0]

    def __getitem__(self, idx):

        if self.valid == True:
            return self.xvalid[idx], self.yvalid[idx]
        else:
            return self.x[idx], self.y[idx]

#Divide the data into batches
dataset = Batch_Taxels(tac_image_train, tac_label_train, tac_image_valid, tac_label_valid)
dataset2 = Batch_Taxels(tac_image_train, tac_label_train, tac_image_valid, tac_label_valid, valid = True)

xelaloader = DataLoader(dataset = dataset, batch_size=32, shuffle=True)
xelaloadervalid = DataLoader(dataset = dataset2, batch_size=32, shuffle=True)

In [57]:
for x, y in xelaloader:
    print (x.shape, y.shape)
    break

torch.Size([32, 3, 4, 4]) torch.Size([32, 1])


In [61]:
inputsize = (3,4,4)
seq_length = 3 
num_layers = 2 
hidden_size = 2
learning_rate = 0.001
num_epochs = 10


class RNN(nn.Module):
    def __init__(self, inputsize, hidden_size, num_layers, num_classes = 1):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(inputsize, hidden_size, num_layers, batch_first = True)
        self.fc  = nn.Linear(hidden_size*seq_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.shape[0], inputsize)
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [62]:
model = RNN(inputsize, hidden_size, num_layers, num_classes = 1)
loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loss = []
valid_loss = []

t_loss = []
v_loss = []

t_acc = []
v_acc = []

t_acc_t = []
v_acc_t = []

TypeError: empty(): argument 'size' must be tuple of SymInts, but found element of type tuple at pos 2

In [63]:
for epoch in range(num_epochs):
    total = 0
    correct = 0
    l_xelaloader = 0

    model.train()
    for (x, y) in (xelaloader):
        #Forward pass
        y_pred = model(x)
       
        #compute the loss
        l = loss(y_pred, y)

        #empty the gradients
        optimizer.zero_grad()

        #compute the gradient
        l.backward()

        #update the weights
        optimizer.step()

        #append each loss per batch
        train_loss.append(l.item())

        #accuracy
        total += y.size(0)
        correct += y_pred.round().eq(y).sum().item()
        l_xelaloader += x.shape[0]
        
    
    t_acc = correct/l_xelaloader
    t_acc_t.append(t_acc)

    total = 0
    correct = 0
    l_xelaloader = 0

    #calculate and plot the validation loss
    model.eval()
    for (x,y) in (xelaloadervalid):
        y_pred_test = model(x)
        lv = loss(y_pred_test, y)
        #append the loss per batch
        valid_loss.append(lv.item())

        #accuracy
        total += y.size(0)
        correct += y_pred_test.round().eq(y).sum().item()
        l_xelaloader += x.shape[0]
        
    v_acc = correct/l_xelaloader
    v_acc_t.append(v_acc)

    #append the total loss and accuracy per epoch
    t_loss.append(np.mean(train_loss))
    v_loss.append(np.mean(valid_loss))

    print(f'For training epoch {epoch+1}, loss ={l:.8f}', f'For validation epoch {epoch+1}, loss ={lv:.8f}'  )

TypeError: zeros(): argument 'size' must be tuple of SymInts, but found element of type tuple at pos 2